In [ ]:
import h5py
import numpy as np
import time
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical

In [ ]:
print(str(time.ctime()) + ": Initializing...")

sarsmerscov_dm_train = h5py.File('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2_dm_train.h5', 'r')
sarsmerscov_dm_val = h5py.File('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2_dm_val.h5', 'r')

sarsmerscov_cm_train = h5py.File('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2_cm_train.h5', 'r')
sarsmerscov_cm_val = h5py.File('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2_cm_val.h5', 'r')

lt = list(open('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/label_train.txt', 'r'))
lv = list(open('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/label_val.txt', 'r')) # open all files

cvae_embeddings = np.load('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2-embeddings.npy', 'r')
cvae_samples = np.load('/gpfs/alpine/gen150/scratch/arjun2612/ORNL_Coding/Code/sars_mers_cov2_dataset/cvae/sars-mers-cov2-samples.npz', 'r')

print(str(time.ctime()) + ": Successfully loaded all data sets!")

In [ ]:
train_size = 60000
val_size = 15000

In [ ]:
label_training = np.array([])
label_validation = np.array([])

for i in range(train_size):
    num = int(str(lt[i]).strip('\n'))
    label_training = np.append(label_training, num)
    
for j in range(val_size):
    num = int(str(lv[j]).strip('\n'))
    label_validation = np.append(label_validation, num)

lt_onehot = to_categorical(label_training) # make one hot vectors
lv_onehot = to_categorical(label_validation)

In [ ]:
trainset_cm = np.array(sarsmerscov_cm_train['contact_maps'][0:train_size]).astype(float) # 60000 x 24 x 24 x 1
valset_cm = np.array(sarsmerscov_cm_val['contact_maps'][0:val_size]).astype(float) # 15000 x 24 x 24 x 1

trainset_cm, label_training = shuffle(trainset_cm, label_training, random_state=0)
valset_cm, label_validation = shuffle(valset_cm, label_validation, random_state=0)

train_3D_cm = np.tril(trainset_cm[:, :, :, 0])
val_3D_cm = np.tril(valset_cm[:, :, :, 0])

In [ ]:
trainset_dm = np.array(sarsmerscov_dm_train['contact_maps'][0:train_size]).astype(float) # 60000 x 24 x 24 x 1
valset_dm = np.array(sarsmerscov_dm_val['contact_maps'][0:val_size]).astype(float) # 15000 x 24 x 24 x 1

trainset_dm, label_training = shuffle(trainset_dm, label_training, random_state=0)
valset_dm, label_validation = shuffle(valset_dm, label_validation, random_state=0)

train_3D_dm = np.tril(trainset_dm[:, :, :, 0])
val_3D_dm = np.tril(valset_dm[:, :, :, 0])

In [ ]:
cvae_embeddings = np.squeeze(cvae_embeddings)[0:val_size]

In [ ]:
lt = None
lv = None
sarsmerscov_dm_train = None
sarsmerscov_dm_val = None 
sarsmerscov_cm_train = None
sarsmerscov_cm_val = None # garbage collection to free up memory

In [ ]:
np.savez('smc2_cm_dataset.npz', train=trainset_cm, val=valset_cm, train3D=train_3D_cm, val3D=val_3D_cm, ltoh=lt_onehot, lvoh=lv_onehot, labtrain=label_training, labval=label_validation)

In [ ]:
np.savez('smc2_dm_dataset.npz', train=trainset_dm, val=valset_dm, train3D=train_3D_dm, val3D=val_3D_dm, ltoh=lt_onehot, lvoh=lv_onehot, labtrain=label_training, labval=label_validation)